# Normalized Difference Water Index (NDWI) 

Articulo de referencia
 
https://www.sciencedirect.com/science/article/abs/pii/S0034425796000673

El Índice de Agua de Diferencia Normalizada (NDWI) fue desarrollado por Gao (1996) como un índice del contenido de agua de la vegetación. El índice es sensible a los cambios en el contenido líquido de los doseles vegetales. Esto significa que el índice se puede utilizar, por ejemplo, para detectar vegetación que experimenta condiciones de sequía o diferenciar los niveles de riego de los cultivos. En áreas secas, los cultivos que se irrigan se pueden diferenciar de la vegetación natural. También se denomina a veces Índice de Humedad de Diferencia Normalizada (NDMI).



**How it Works:**
 
**Healthy Vegetation:**
- High NIR reflectance (due to leaf structure).
- Low SWIR reflectance (water strongly absorbs SWIR).
- Result: Positive NDWI values indicate high water content.
 
**Stressed or Dry Vegetation:**

- NIR reflectance decreases slightly, but SWIR reflectance increases significantly (less water absorption).
- Result: NDWI values decrease, approaching zero or negative, indicating low water content.


**Interpretation:**
- High NDWI (Near 1) – Vegetation with high water content (healthy).
- Moderate NDWI (0 to 0.5) – Vegetation with moderate water content.
- Low NDWI (Negative values) – Stressed or dry vegetation


**Applications:**
- Agriculture – Monitor crop water stress and optimize irrigation.
- Forestry – Assess drought conditions in forests and detect forest fires.
- Climate Studies – Track large-scale vegetation response to climate change.
- Disaster Management – Identify areas of vegetation at risk of drought or fire.

La formula del NDWI es la siguiente: 

<div style="text-align: center;">
    <img src="Images/NDWI formula.png" alt="Descripción de la imagen" width="500">
</div>
 

**BANDAS NECESARIAS:** 
 
En este caso utilizaremos imagenes de Sentinel 2. Por ende, hay que encontrar cuales son las bandas en sentinel 2 que corresponden a estas longitudes de onda. 

En el caso de Sentinel 2 la banda NIR (Near Infrared) es la banda **8** con una longitud de onda de Central Wavelength = 842nm.  Por otra parte, la banda de SWIR (short-wave infrared) es mejor el uso de la banda 11 que tiene un centro de longitud de onda  = 1610nm. 

**NOTA:** El rnago de SWIR adecuados aparentmenete son entre 1,400 a 1,900 nm. 



**IMPORTACION DE LIBRERIAS**

In [1]:
import os 
import ee 
import geemap
import pandas as pd 
import requests

**AUTENTIFICACION**

In [2]:
# 1) Obtención de la dirección de trabajo 
direction = os.getcwd()

# Remove the last part of the path
direction = os.path.dirname(direction)

# Dirección de la llave 
service_account = direction + '/conf/local/gcp-for-data-science-397913-4fd843feede1.json'

# Autentificación 
credentials = ee.ServiceAccountCredentials(email=None, key_file=service_account)
ee.Initialize(credentials)

## 1) Lectura y procesamiento de imagenes  
Las imagenes satelitales se almacenaron en un bucket de GCP, por lo que serán leídas de ahi mismo.

API Reference:
 
  https://developers.google.com/earth-engine/apidocs/ee-image-loadgeotiff

### 1.1) Cálculo y exportación de imagenes con NDWI

In [12]:
# 1) Lectura del nombre de las imagenes en el bucket 
images_names = pd.read_csv(filepath_or_buffer=direction + "/data/02_intermediate/sentinel_images_names.csv")

# Diccionario vacio, aqui almacenaremos los resultados 
NDWI ={}

# Creación del mapa base 
Map = geemap.Map()

# Path to the image in the GCS bucket
image_center = images_names["Images names"][0]
image_center   = f'gs://earth_engine_selene/sentinel2/Row_Data/{image_center}.tif'

# Load the image from the GCS bucket
image = ee.Image.loadGeoTIFF(uri=image_center)

# Central el mapa 
Map.centerObject(ee_object=image, zoom= 9)

# 2) Inicio del ciclo 
for i in images_names["Images names"]:

    # 1) Descarga de imagenes
    # Path to the image in the GCS bucket
    gcs_path = f'gs://earth_engine_selene/sentinel2/Row_Data/{i}.tif' # Crear más bien un GCP path base

    # Load the image from the GCS bucket
    image = ee.Image.loadGeoTIFF(gcs_path)

    # 2) Calculate NDVI (if bands are organized)
    image = image.normalizedDifference(bandNames=['B8', 'B11']).rename('NDWI')  # Renombre de la banda resultante (capa)
    print(f"NDWI Calculated for {i}")
    
    # Define visualization parameters
    vis_params = {
        'min': -0.5,
        'max': 1,
        'palette': ['brown', 'yellow', 'green']
    }

    # 4) Descarga de la imagen a PNG
    # Obtener el url de la imagen
    url = image.getThumbURL(params=vis_params)
    print(f"Thumbnail URL for {i}: {url}")

    #Download the image and save it locally
    output_file = direction + f"/data/07_model_output/Sentinel/NDWI/Row_Results/NDWI_{i}.png"
    
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print(f"Image successfully downloaded as '{output_file}'")
    else:
        print(f"Error: Unable to download the image. HTTP status code {response.status_code}")
        
    # 5) Almacenamiento de resultados en un diccionario 
    NDWI[i] = image

    # 6) Exportación de las imágen procesado geotiff al bucket de GCP, está imagen ya cotiene el cálculo de NDWI
    # Indicamos la tarea de exportacion
    task = ee.batch.Export.image.toCloudStorage(
    image=image,
    #description=f"Export", # Este es el nombre del archivo que se exportara , si vas a usar el filenameprexi no es necesario esto
    bucket='earth_engine_selene',  # Cambia esto al nombre de tu bucket
    fileNamePrefix=f"sentinel2/NDWI/NDWI_{i}", # Damos una dirección dentro del bucket y le damos nombre
    scale=10,  # Resolución en metros
    fileFormat='GeoTIFF')  # Formato de archivo

    # Inicio de la exportación
    task.start()  # Inicia la tarea de exportación
    print("Se exportó la imagen a Cloud Storage:", i)

    # 7) Añadimos la imagen al mapa html
    Map.addLayer(ee_object=image,  vis_params= vis_params, name=f'NDWI {i}', shown=False)
    
# Exportación del HTML 
html_file = direction + "/data/07_model_output/Sentinel/NDWI/Map_NDWI.html"
Map.to_html(filename=html_file, title="My Map", width="100%", height="880px") # Exporacion a una html

NDWI Calculated for Image_2024-10-04 00:00:00
Thumbnail URL for Image_2024-10-04 00:00:00: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/b17df706def3c61b29ca0983bc6dd26f-cba280bf441f23066ea6266e0963baf3:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/Sentinel/NDWI/Row_Results/NDWI_Image_2024-10-04 00:00:00.png'
Se exportó la imagen a Cloud Storage: Image_2024-10-04 00:00:00
NDWI Calculated for Image_2024-10-14 00:00:00
Thumbnail URL for Image_2024-10-14 00:00:00: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/36e06b642ef32f28657ce65bf1013c6d-19ce974f124fe86e30bbc7acf7090541:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/Sentinel/NDWI/Row_Results/NDWI_Image_2024-10-14 00:00:00.png'
Se exportó la imagen a Cloud Storage: Image_2024-10-14 00:00:00
NDWI Calculated for Image_2024-10-

### 2) Post procesamiento
En esta sección  lo que se busca es  tomas los geotiff generados del NDVI y generar un mapa a partir de estos. El mapa contendrá los siguientes elementos adicionales:

- Titulo
- Fecha de la imagen
- Leyenda con el valor de los colores
- Escala de la imagen

Para esto se utilizarán 2 librerías adicionales las cuales son: **Rasterio y matplolib**.

[Rasterio tiene un modulo de visualización (plot)](https://rasterio.readthedocs.io/en/stable/topics/plotting.html) que se convina con matplotlib para la visualización de archivos tipo raster

**IMPORTANTE:** Para que rasterio pueda hacer la lectura directamente desde el bucket es necesario que haga la autentificación con mi service account en variasbles del sistema.


In [13]:
# Autentificacion , variables del sistema
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =  service_account

In [15]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
from matplotlib_scalebar.scalebar import ScaleBar  # <-- for the scale bar

# Definir la paleta de colores personalizada
palette = [
    "#FF0000", "#FF1700", "#FF2E00", "#FF4500", "#FF6500", "#FF8500",
    "#FFA500", "#FAC008", "#F6D80F", "#F1F117", "#D1ED19", "#B1E91B",
    "#91E51D", "#6FE120", "#53C918", "#37B110", "#1B9908", "#008000"
]

cmap = ListedColormap(colors=palette) # Esto crea una lista de colores que puede ser integrado directamente en la figura

# Fijar el rango de valores de -1 a 1 para todas las imágenes
norm = Normalize(vmin=-1, vmax=1) # IMPORTANTE: Hay que confirmar si es necesario que las imagenes las mostremos de -1 a 1. Por lo que entiendo de los valores de NDVI en las plantas su valores realmente van desde cercano de 0 a 1. Los valores negativos serán asociados a otras superfices. Por lo que no veo necesario inclurilas dentro de la escala (los valores negativos).

# Suponiendo que 'images_names' es un DataFrame o diccionario con la llave "Images names"
# Y que 'direction' es una variable con la ruta de salida
for i in images_names["Images names"]:
    print(i)

    # Ruta del archivo TIFF
    tiff_path = f"gs://earth_engine_selene/sentinel2/NDWI/NDWI_{i}.tif"

    # Lectura del archivo
    with rasterio.open(tiff_path) as src:
        # Lectura del archivo
        raster_data = src.read(1) # Creo que aqui se selecciona la banda de la imagen, hay que confirmar

        fig, ax = plt.subplots(figsize=(8, 6))

        # Plot del raster con la paleta personalizada y escala fija
        img = ax.imshow(raster_data, cmap=cmap, norm=norm)

        # Añadir una barra de colores (leyenda de NDVI)
        cbar = fig.colorbar(img, ax=ax)
        cbar.set_label('NDWI Value')

        # -------------------------------------------------
        # AÑADIR LA BARRA DE ESCALA (ScaleBar)
        # -------------------------------------------------
        # (1) Obtener la resolución de pixel (asumiendo un CRS en metros, por ejemplo UTM)
        pixel_size_x = abs(src.transform[0])  # tamaño de pixel en X (ej. metros)

        # (2) Crear y añadir la ScaleBar
        scalebar = ScaleBar(
            pixel_size_x,
            units='m',         # 'm' para metros, 'km' si desea kilómetros
            location='lower left',  # use espacio, no subrayado (p.ej. "lower left")
            length_fraction=0.25    # porcentaje del ancho del eje que ocupa la scale bar
        )
        ax.add_artist(scalebar)

        # Quitar los valores de los ejes X e Y
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')

        # Añadir título
        ax.set_title(f"NDWI Map {i}")

        # Guardar la figura en un archivo .png
        fig.savefig(direction + f"/data/07_model_output/Sentinel/NDWI/Final_Results/NDVI_{i}.png")

        # Cerrar la figura para no mostrarla en pantalla
        plt.close(fig)

Image_2024-10-04 00:00:00
Image_2024-10-14 00:00:00
Image_2024-10-19 00:00:00
Image_2024-10-24 00:00:00
Image_2024-10-29 00:00:00
Image_2024-11-03 00:00:00
Image_2024-11-08 00:00:00
Image_2024-11-13 00:00:00
Image_2024-11-18 00:00:00
Image_2024-11-23 00:00:00
Image_2024-11-28 00:00:00
Image_2024-12-03 00:00:00
Image_2024-12-18 00:00:00
Image_2024-12-23 00:00:00
Image_2024-12-28 00:00:00
Image_2025-01-02 00:00:00
